#### Import Package

In [1]:
import talib
import wrds

from core.operation import *
from class_model.model_prep import ModelPrep

#### Check permno

In [ ]:
per_hist = read_stock(get_large(False) / 'permno_to_train_fund.csv')
print(f"Length of permno_to_train_fund: {len(per_hist)}")

per_live = read_stock(get_large(True) / 'permno_live.csv')
print(f"Length of permno_common: {len(per_live)}")

In [ ]:
print(f"per_live equal to per_hist? {per_live == per_hist}")

In [ ]:
for permno in per_live:
    if permno not in per_hist:
        print(permno)

#### Check price

In [ ]:
price_hist = pd.read_parquet(get_parquet(False) / 'data_price.parquet.brotli')
price_hist_pre = get_stocks_data(price_hist, per_hist)
price_hist_pre = set_timeframe(price_hist_pre, '2008-01-01', '2023-01-01')

price_crsp = pd.read_parquet(get_parquet(True) / 'data_crsp_price.parquet.brotli')

price_live = pd.read_parquet(get_parquet(True) / 'data_price.parquet.brotli')
price_live_pre = set_timeframe(price_live, '2008-01-01', '2023-01-01')

In [ ]:
print(f"price_hist equal to price_crsp? {price_hist.equals(price_crsp)}")
print(f"price_hist equal to price_crsp? {price_hist_pre.equals(price_live_pre)}")

#### Check misc

In [ ]:
misc_live = pd.read_parquet(get_parquet(True) / 'data_misc.parquet.brotli')

#### Check ind

In [67]:
ind_hist = pd.read_parquet(get_parquet(False) / 'data_ind.parquet.brotli')
ind_live = pd.read_parquet(get_parquet(True) / 'data_ind.parquet.brotli')
ind_hist.columns = ['ind_hist']
ind_live.columns = ['ind_live']
ind_comb = ind_hist.join(ind_live)
ind_comb.loc[ind_comb.ind_hist != ind_comb.ind_live]

ind_hist  ind_live
permno date                          
12017  2010-07-27  451030.0  253020.0
       2010-07-28  451030.0  253020.0
       2010-07-29  451030.0  253020.0
       2010-07-30  451030.0  253020.0
       2010-08-02  451030.0  253020.0
...                     ...       ...
92665  2020-10-30  352010.0  352020.0
       2020-11-02  352010.0  352020.0
       2020-11-03  352010.0  352020.0
       2020-11-04  352010.0  352020.0
       2020-11-05  352010.0  352020.0

[91377 rows x 2 columns]

In [65]:
unique_ind_hist_counts = ind_comb.groupby(level='permno')['ind_hist'].nunique()
permnos_with_multiple_ind_hist = unique_ind_hist_counts[unique_ind_hist_counts > 1].index.tolist()
print(permnos_with_multiple_ind_hist)
unique_ind_live_counts = ind_comb.groupby(level='permno')['ind_live'].nunique()
permnos_with_multiple_ind_live = unique_ind_live_counts[unique_ind_live_counts > 1].index.tolist()
print(permnos_with_multiple_ind_live)

[12017, 13889, 14686, 14704, 14811, 15177, 15436, 16656, 18148, 24643, 45356, 63546, 76963, 77437, 79678, 82530, 85592, 85909, 86165, 89003, 90640, 91575, 92032, 92252, 92665]
[]


In [68]:
ind_fama_hist = pd.read_parquet(get_parquet(False) / 'data_ind_fama.parquet.brotli')
ind_fama_live = pd.read_parquet(get_parquet(True) / 'data_ind_fama.parquet.brotli')
ind_fama_hist.columns = ['ind_hist']
ind_fama_live.columns = ['ind_live']
ind_fama_comb = ind_fama_hist.join(ind_fama_live)
ind_fama_comb.loc[ind_fama_comb.ind_hist != ind_fama_comb.ind_live]

ind_hist  ind_live
permno date                          
10145  2012-01-31         3         4
       2012-02-01         3         4
       2012-02-02         3         4
       2012-02-03         3         4
       2012-02-06         3         4
...                     ...       ...
93436  2022-12-23         4        23
       2022-12-27         4        23
       2022-12-28         4        23
       2022-12-29         4        23
       2022-12-30         4        23

[1682524 rows x 2 columns]

#### Check prep

In [16]:
per_hist = read_stock(get_large(False) / 'permno_to_train_fund.csv')
print(f"Length of permno_to_train_fund: {len(per_hist)}")

per_live = read_stock(get_large(True) / 'permno_live.csv')
print(f"Length of permno_common: {len(per_live)}")

Length of permno_to_train_fund: 1054
Length of permno_common: 1054


In [78]:
start_model = '2008-01-01'
current_date = '2023-01-01'
factor_name = 'factor_talib'

In [79]:
factor_hist = ModelPrep(live=False, factor_name=factor_name, group='permno', interval='D', kind='price', stock=per_hist, div=False, start=start_model, end=current_date, save=False).prep()
factor_live = ModelPrep(live=True, factor_name=factor_name, group='permno', interval='D', kind='price', stock=per_live, div=False, start=start_model, end=current_date, save=False).prep()

Creating factor_talib -------------------------------------- | ✔
Shape: (3114791, 20)
Creating factor_talib -------------------------------------- | ✔
Shape: (3114791, 20)


In [80]:
factor_hist.equals(factor_live)

True

#### Check talib

In [49]:
price_hist = pd.read_parquet(get_parquet(False) / 'data_price.parquet.brotli')
price_hist_pre = get_stocks_data(price_hist, per_hist)
price_hist_pre = set_timeframe(price_hist_pre, '2008-01-01', '2023-01-01')

price_crsp = pd.read_parquet(get_parquet(True) / 'data_crsp_price.parquet.brotli')

price_live = pd.read_parquet(get_parquet(True) / 'data_price.parquet.brotli')
price_live = set_timeframe(price_live, '2008-01-01', '2025-01-01')
price_live_pre = set_timeframe(price_live, '2008-01-01', '2023-01-01')

price_hist_pre['ad'] = (price_hist_pre.groupby('permno', group_keys=False).apply(lambda x: talib.AD(x.High, x.Low, x.Close, x.Volume) / x.expanding().Volume.mean()))
price_live_pre['ad'] = (price_live_pre.groupby('permno', group_keys=False).apply(lambda x: talib.AD(x.High, x.Low, x.Close, x.Volume) / x.expanding().Volume.mean()))
price_live['ad'] = (price_live.groupby('permno', group_keys=False).apply(lambda x: talib.AD(x.High, x.Low, x.Close, x.Volume) / x.expanding().Volume.mean()))
price_live_test = set_timeframe(price_live, '2008-01-01', '2023-01-01')

print(f"price_hist_pre equals price_live_pre? {price_hist_pre.equals(price_live_pre)}")
print(f"price_live_test equals price_hist_pre? {price_live_test.equals(price_hist_pre)}")

price_hist_pre equals price_live_pre? True
price_live_test equals price_hist_pre? True


#### Check WRDS

In [72]:
with open(get_config() / 'api_key.json') as f:
    config = json.load(f)
    wrd_key = config['wrd_key']
    fred_key = config['fred_key']

db = wrds.Connection(wrds_username=wrd_key)

Loading library list...
Done


In [73]:
db.list_tables(library='crspa')

NotSubscribedError: You do not have permission to access the crspa library